In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [21]:
with open('/Users/rijavoloder/.secret/yelp_api.json') as f:  
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [22]:
LOCATION = 'San Diego, CA,92103'
TERM = 'Tacos'

In [23]:
JSON_FILE = f"Data/results_in_progress_tacos.json"
JSON_FILE

'Data/results_in_progress_tacos.json'

In [24]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    if len(folder)>0:
        os.makedirs(folder,exist_ok=True)
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    with open(JSON_FILE,'w') as f:
        json.dump([],f) 
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_tacos.json not found. Saving empty list to file.


In [25]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [26]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [27]:
total_results = results['total']
total_results

1600

In [28]:
results_per_page = len(results['businesses'])
results_per_page

20

In [29]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

80

In [30]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [31]:
import time
for i in tqdm_notebook(range(n_pages)):
    time.sleep(.2)

  0%|          | 0/80 [00:00<?, ?it/s]

In [32]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    previous_results.extend(results['businesses'])
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/80 [00:00<?, ?it/s]

YelpAPIError: VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.

In [33]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,EGj92CH6mWFiRvHB1zyMAw,lucha-libre-gourmet-taco-shop-san-diego,Lucha Libre Gourmet Taco Shop,https://s3-media4.fl.yelpcdn.com/bphoto/Gn4lpp...,False,https://www.yelp.com/biz/lucha-libre-gourmet-t...,8105,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.0,"{'latitude': 32.743229, 'longitude': -117.181628}","[delivery, pickup]",$$,"{'address1': '1810 W Washington St', 'address2...",+16192968226,(619) 296-8226,1094.036360
1,wL7Z4NJakeQ4yz9GSWki0A,ortizs-taco-shop-san-diego,Ortiz's Taco Shop,https://s3-media2.fl.yelpcdn.com/bphoto/mo9e1M...,False,https://www.yelp.com/biz/ortizs-taco-shop-san-...,656,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 32.74049, 'longitude': -117.22903}",[delivery],$,"{'address1': '3704 Voltaire St', 'address2': '...",+16192224476,(619) 222-4476,5514.722095
2,ssJJPFuV6tTDgHYe2w9csQ,oscars-mexican-seafood-san-diego-3,Oscars Mexican Seafood,https://s3-media1.fl.yelpcdn.com/bphoto/nmtXm1...,False,https://www.yelp.com/biz/oscars-mexican-seafoo...,3164,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.5,"{'latitude': 32.808189, 'longitude': -117.260846}","[delivery, pickup]",$,"{'address1': '703 Turquoise St', 'address2': N...",+18584886392,(858) 488-6392,10978.546115
3,jjkHyoBlzNMnHJNXwyxsJQ,the-taco-stand-san-diego-6,The Taco Stand,https://s3-media3.fl.yelpcdn.com/bphoto/yvKAkw...,False,https://www.yelp.com/biz/the-taco-stand-san-di...,637,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",4.0,"{'latitude': 32.7411646431234, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '3000 Upas St', 'address2': 'Ste ...",+16197958797,(619) 795-8797,3820.802448
4,c2iFpIROV9OVK3wd24J0Tw,la-vecindad-neighborhood-tacos-san-diego-2,La Vecindad Neighborhood Tacos,https://s3-media3.fl.yelpcdn.com/bphoto/Cm9oF2...,False,https://www.yelp.com/biz/la-vecindad-neighborh...,1171,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 32.7476563, 'longitude': -117.160...",[delivery],$$,"{'address1': '3827 5th Ave', 'address2': '', '...",+16195012885,(619) 501-2885,978.184779


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,RtReZMiKX5lw7PRt76w6xg,beaumonts-la-jolla-2,Beaumont's,https://s3-media2.fl.yelpcdn.com/bphoto/X21oO0...,False,https://www.yelp.com/biz/beaumonts-la-jolla-2?...,719,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 32.81503, 'longitude': -117.27065}",[delivery],$$,"{'address1': '5662 La Jolla Blvd', 'address2':...",+18584590474,(858) 459-0474,12169.155575
996,1gs4dAufjVCWH9vpRSmblA,el-pollo-loco-san-diego-15,El Pollo Loco,https://s3-media2.fl.yelpcdn.com/bphoto/64Pu4U...,False,https://www.yelp.com/biz/el-pollo-loco-san-die...,105,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.0,"{'latitude': 32.76997834510157, 'longitude': -...",[delivery],$,"{'address1': '2017 Camino Del Este', 'address2...",+16196881189,(619) 688-1189,3743.852801
997,AewH2IkKMYO6pmQKz1x7vQ,barefoot-bar-and-grill-san-diego,Barefoot Bar & Grill,https://s3-media4.fl.yelpcdn.com/bphoto/4uTm_c...,False,https://www.yelp.com/biz/barefoot-bar-and-gril...,704,"[{'alias': 'newamerican', 'title': 'American (...",3.5,"{'latitude': 32.7741342997751, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '1404 Vacation Rd', 'address2': '...",+18585815960,(858) 581-5960,7299.931004
998,nhRT2MHx3nRi6olICNHIEg,san-diego-pier-cafe-san-diego,San Diego Pier Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/xhEt2d...,False,https://www.yelp.com/biz/san-diego-pier-cafe-s...,850,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.0,"{'latitude': 32.7112551, 'longitude': -117.170...",[delivery],$$,"{'address1': '885 W Harbor Dr', 'address2': ''...",+16192393968,(619) 239-3968,4049.464766
999,D3RkEG1FblpTWbr7AnGCuA,palominos-mexican-and-seafood-san-diego-2,Palominos Mexican & Seafood,https://s3-media3.fl.yelpcdn.com/bphoto/VHBfi8...,False,https://www.yelp.com/biz/palominos-mexican-and...,264,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",3.5,"{'latitude': 32.830780507853056, 'longitude': ...",[delivery],$,"{'address1': '9353 Clairemont Mesa Blvd', 'add...",+18582928470,(858) 292-8470,10286.474158


In [34]:
final_df.duplicated(subset='id').sum()

3

In [35]:
final_df.to_csv('Data/final_results_crab_cakes.csv.gz', compression='gzip',index=False)